# VectorStore-Backed Memory

`VectorStoreRetrieverMemory` stores memories in a VectorDB and queries the top-K most "salient" docs every time it is called.

This differs from most of the other Memory classes in that it doesn't explicitly track the order of interactions.

In [1]:
from datetime import datetime
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.memory import VectorStoreRetrieverMemory
from langchain.chains import ConversationChain

### Initialize your VectorStore

Depending on the store you choose, this step may look different. Consult the relevant VectorStore documentation for more details.

In [2]:
import faiss

from langchain.docstore import InMemoryDocstore
from langchain.vectorstores import FAISS


embedding_size = 1536 # Dimensions of the OpenAIEmbeddings
index = faiss.IndexFlatL2(embedding_size)
embedding_fn = OpenAIEmbeddings().embed_query
vectorstore = FAISS(embedding_fn, index, InMemoryDocstore({}), {})

### Create your the VectorStoreRetrieverMemory

The memory object is instantiated from any VectorStoreRetriever.

In [3]:
# In actual usage, you would set `k` to be a higher value, but we use k=1 to show that
# the vector lookup still returns the semantically relevant information
retriever = vectorstore.as_retriever(search_kwargs=dict(k=1))
memory = VectorStoreRetrieverMemory(retriever=retriever)

# When added to an agent, the memory object can save pertinent information from conversations or used tools
memory.save_context({"input": "check the latest scores of the Warriors game"}, {"output": "the Warriors are up against the Astros 88 to 84"})
memory.save_context({"input": "I need help doing my taxes - what's the standard deduction this year?"}, {"output": "..."})
memory.save_context({"input": "What's the the time?"}, {"output": f"It's {datetime.now()}"}) # 

In [4]:
# Notice the first result returned is the memory pertaining to tax help, which the language model deems more semantically relevant
# to a 1099 than the other documents, despite them both containing numbers.
print(memory.load_memory_variables({"prompt": "What's a 1099?"})["history"])

input: I need help doing my taxes - what's the standard deduction this year?
output: ...


## Using in a chain
Let's walk through an example, again setting `verbose=True` so we can see the prompt.

In [5]:
llm = OpenAI(temperature=0) # Can be any valid LLM
conversation_with_summary = ConversationChain(
    llm=llm, 
    # We set a very low max_token_limit for the purposes of testing.
    memory=memory,
    verbose=True
)
conversation_with_summary.predict(input="Hi, my name is Perry, what's up?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
input: I need help doing my taxes - what's the standard deduction this year?
output: ...
Human: Hi, my name is Perry, what's up?
AI:

> Finished chain.


" Hi Perry, my name is AI. I'm doing great, how about you? I understand you need help with your taxes. What specifically do you need help with?"

In [6]:
# Here, the basketball related content is surfaced
conversation_with_summary.predict(input="If the Cavaliers were to face off against the Warriers or the Astros, who would they most stand a chance to beat?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
input: check the latest scores of the Warriors game
output: the Warriors are up against the Astros 88 to 84
Human: If the Cavaliers were to face off against the Warriers or the Astros, who would they most stand a chance to beat?
AI:

> Finished chain.


" It's hard to say without knowing the current form of the teams. However, based on the current scores, it looks like the Cavaliers would have a better chance of beating the Astros than the Warriors."

In [7]:
# Even though the language model is stateless, since relavent memory is fetched, it can "reason" about the time.
# Timestamping memories and data is useful in general to let the agent determine temporal relevance
conversation_with_summary.predict(input="What day is it tomorrow?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
input: What's the the time?
output: It's 2023-04-13 09:18:55.623736
Human: What day is it tomorrow?
AI:

> Finished chain.


' Tomorrow is 2023-04-14.'

In [8]:
# The memories from the conversation are automatically stored,
# since this query best matches the introduction chat above,
# the agent is able to 'remember' the user's name.
conversation_with_summary.predict(input="What's your name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
input: Hi, my name is Perry, what's up?
response:  Hi Perry, my name is AI. I'm doing great, how about you? I understand you need help with your taxes. What specifically do you need help with?
Human: What's your name?
AI:

> Finished chain.


" My name is AI. It's nice to meet you, Perry."